# Creation of Acquired Nodes

In [47]:
import pandas as pd
acquisition_table = pd.read_csv('Acquisitions_Table.csv')

In [48]:
acquisition_table = acquisition_table[['Date','Company','Business','Country','Value (USD)']]
acquisition_table

,Date,Company,Business,Country,Value (USD)
0,"September 24, 1993",Crescendo Communications,LAN switching,United States,"$94,500,000"
1,"July 12, 1994",Newport Systems Solutions,Router,United States,"$95,000,000"
2,"October 24, 1994",Kalpana,LAN switching,United States,"$204,000,000"
3,"December 8, 1994",LightStream,ATM switching,United States,"$120,000,000"
4,"August 10, 1995",Combinet,Remote desktop software,United States,"$114,200,000"
...,...,...,...,...,...
202,"December 18, 2018",Luxtera,Privately-held semiconductor company,United States,----
203,"December 18, 2018",Singularity Networks,Network Analytics Company,United States,----
204,"June 6, 2019",Sentryo,Cybersecurity for the Industrial Internet,France,----
205,"July 9, 2019",Acacia Communications,Optical Interconnects,United States,$2.6×109


In [49]:
# replacing inconsistensies in Value of acquisition column
for i in range(0,len(acquisition_table)):
    acquisition_table['Value (USD)'][i] = str(acquisition_table['Value (USD)'][i]).replace('----','-')
    acquisition_table['Value (USD)'][i] = str(acquisition_table['Value (USD)'][i]).replace('---','-')
    acquisition_table['Value (USD)'][i] = acquisition_table['Value (USD)'][i].replace('$','')
    acquisition_table['Value (USD)'][i] = acquisition_table['Value (USD)'][i].replace('-','nan')
    acquisition_table['Value (USD)'][i] = acquisition_table['Value (USD)'][i].replace('—','nan')
    acquisition_table['Value (USD)'][i] = acquisition_table['Value (USD)'][i].replace(',','')
    if acquisition_table['Value (USD)'][i].find('×') != -1:
        loc = acquisition_table['Value (USD)'][i].find('×')
        multiplier = acquisition_table['Value (USD)'][i][-1]
        acquisition_table['Value (USD)'][i] = int(float(acquisition_table['Value (USD)'][i][:loc])*(10**int(multiplier)))
    acquisition_table['Value (USD)'][i] = float(acquisition_table['Value (USD)'][i])
acquisition_table

,Date,Company,Business,Country,Value (USD)
0,"September 24, 1993",Crescendo Communications,LAN switching,United States,9.45e+07
1,"July 12, 1994",Newport Systems Solutions,Router,United States,9.5e+07
2,"October 24, 1994",Kalpana,LAN switching,United States,2.04e+08
3,"December 8, 1994",LightStream,ATM switching,United States,1.2e+08
4,"August 10, 1995",Combinet,Remote desktop software,United States,1.142e+08
...,...,...,...,...,...
202,"December 18, 2018",Luxtera,Privately-held semiconductor company,United States,NaN
203,"December 18, 2018",Singularity Networks,Network Analytics Company,United States,NaN
204,"June 6, 2019",Sentryo,Cybersecurity for the Industrial Internet,France,NaN
205,"July 9, 2019",Acacia Communications,Optical Interconnects,United States,2.6e+09


In [50]:
# creating node properties table for Neo4j insertion
node_properties= pd.DataFrame(columns=['Node_Primary_Key','Node_Type_Key','Property_Name','Property'])
node_properties

,Node_Primary_Key,Node_Type_Key,Property_Name,Property


In [51]:
# creating edge properties table for Neo4j insertion
edge_properties = pd.DataFrame(columns = ['Edge_Primary_Key','Edge_Type_Key','Property_Name','Property'])
edge_properties

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property


In [52]:
#creating rows for both edge and Node properties table for Neo4j
for i in range(0,len(acquisition_table)):
    t = {'Node_Primary_Key': [i, i, i, i], 'Node_Type_Key': [3, 3, 3, 3], 
         'Property_Name' : ['Name', 'Date', 'Value (USD)', 'Country'], 
         'Property' : [acquisition_table['Company'][i], acquisition_table['Date'][i], 
                       acquisition_table['Value (USD)'][i], acquisition_table['Country'][i][1:]]}
    temp_node = pd.DataFrame(data = t)
    e = {'Edge_Primary_Key': [i], 'Edge_Type_Key': [3], 'Property_Name': ['Reason'], 'Property': [acquisition_table['Business'][i].lower()]}
    temp_edge = pd.DataFrame(data = e)
    node_properties = node_properties.append(temp_node, ignore_index = True)
    edge_properties = edge_properties.append(temp_edge, ignore_index = True)

In [55]:
edge_properties

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property
0,0,3,Reason,lan switching
1,1,3,Reason,router
2,2,3,Reason,lan switching
3,3,3,Reason,atm switching
4,4,3,Reason,remote desktop software
...,...,...,...,...
202,202,3,Reason,privately-held semiconductor company
203,203,3,Reason,network analytics company
204,204,3,Reason,cybersecurity for the industrial internet
205,205,3,Reason,optical interconnects


In [56]:
node_properties

,Node_Primary_Key,Node_Type_Key,Property_Name,Property
0,0,3,Name,Crescendo Communications
1,0,3,Date,"September 24, 1993"
2,0,3,Value (USD),9.45e+07
3,0,3,Country,United States
4,1,3,Name,Newport Systems Solutions
...,...,...,...,...
823,205,3,Country,United States
824,206,3,Name,Voicea
825,206,3,Date,"Aug 6, 2019"
826,206,3,Value (USD),NaN


In [57]:
edge_properties.to_pickle("acquisition_edge_properties.pkl")
node_properties.to_pickle("acquisition_node_properties.pkl")

In [58]:
node_properties['Property'][3]

'United States'